In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
import matplotlib.pyplot as plt
import scipy.stats as stats
import json
from collections import Counter
import random
import seaborn as sns

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


**Safegraph**

In [ ]:
#Import places
safegraph_ma_pla = pd.read_csv ('drive/MyDrive/Amenities/data/MA_places.csv', index_col = False)
safegraph_ma_pla.drop('Unnamed: 0', axis=1, inplace=True)
# Erase the prefix
safegraph_ma_pla.columns = safegraph_ma_pla.columns.str.replace('sg_c__','')
safegraph_ma_pla.head(3)

,placekey,parent_placekey,location_name,safegraph_brand_ids,brands,top_category,sub_category,naics_code,latitude,longitude,...,region,postal_code,iso_country_code,phone_number,open_hours,category_tags,opened_on,closed_on,tracking_closed_since,geometry_type
0,222-222@629-2fq-whq,zzw-222@629-2fn-bp9,Subway,SG_BRAND_de80593878cb1673c62a7f338dc7e4e1,Subway,Restaurants and Other Eating Places,Limited-Service Restaurants,722513.0,42.193087,-72.551128,...,MA,1022,US,NaN,"{ ""Mon"": [[""9:00"", ""16:00""]], ""Tue"": [[""9:00"",...","Counter Service,Sandwich Shop,Lunch,Breakfast,...",NaN,NaN,2019-07-01,POLYGON
1,222-222@629-2ft-f2k,zzw-222@629-2fn-bp9,Hampden County Pre Release Center,NaN,NaN,"Justice, Public Order, and Safety Activities",Correctional Institutions,922140.0,42.201540,-72.509248,...,MA,1056,US,1.413858e+10,NaN,NaN,NaN,NaN,2019-07-01,POLYGON
2,222-222@629-2ht-wff,NaN,Pop'n Kork,NaN,NaN,"Beer, Wine, and Liquor Stores","Beer, Wine, and Liquor Stores",445310.0,42.168252,-72.488897,...,MA,1056,US,NaN,NaN,NaN,NaN,NaN,2019-07-01,POLYGON


In [ ]:
len(safegraph_ma_pla)

171222

In [ ]:
#Import patterns
safegraph_ma_pat = pd.read_csv (r'drive/MyDrive/Amenities/data/MA_patterns.csv', index_col = False)
safegraph_ma_pat.drop('Unnamed: 0', axis=1, inplace=True)

#Merge patterns & places
safegraph_ma = safegraph_ma_pla.merge(safegraph_ma_pat, how='right',left_on='placekey', right_on='placekey')

# Erase the suffix
safegraph_ma.columns = safegraph_ma.columns.str.replace('_x','')

#del safegraph_ma_pat, safegraph_ma_pla
safegraph_ma.head(2)

,placekey,parent_placekey,location_name,safegraph_brand_ids,brands,top_category,sub_category,naics_code,latitude,longitude,...,related_same_day_brand,related_same_month_brand,popularity_by_hour,popularity_by_day,device_type,normalized_visits_by_state_scaling,normalized_visits_by_region_naics_visits,normalized_visits_by_region_naics_visitors,normalized_visits_by_total_visits,normalized_visits_by_total_visitors
0,zzy-222@65y-y8f-kpv,222-222@65y-y8f-j7q,Berkshire Community College,NaN,NaN,"Colleges, Universities, and Professional Schools","Colleges, Universities, and Professional Schools",611310.0,42.459515,-73.316426,...,"{""Walmart"":8,""McDonald's"":7,""Dunkin'"":7,""Price...","{""Dunkin'"":70,""Walmart"":60,""McDonald's"":56,""Pr...","[0,0,0,0,0,0,0,43,67,89,94,84,75,68,57,35,11,2...","{""Monday"":30,""Tuesday"":45,""Wednesday"":53,""Thur...","{""android"":18,""ios"":38}",4818.356801,0.000963,0.005545,0.000009,0.000008
1,zzy-222@62j-ynz-249,NaN,Mill Stores,NaN,NaN,Furniture Stores,Furniture Stores,442110.0,41.666426,-70.135151,...,"{""Stop & Shop"":11,""Alltown"":7,""Mobil"":7,""Dunki...","{""Dunkin'"":71,""Walmart"":54,""CVS"":46,""Stop & Sh...","[5,5,5,5,5,5,5,5,2,1,1,9,8,2,2,4,1,0,8,9,5,4,5,5]","{""Monday"":2,""Tuesday"":3,""Wednesday"":1,""Thursda...","{""android"":15,""ios"":11}",803.059467,0.000806,0.001272,0.000002,0.000003


In [ ]:
# Filter just the columns believed to be useful
columns_keep = ['placekey', 'location_name', 'top_category', 'sub_category', 'naics_code', 'latitude',
                'longitude','raw_visit_counts','raw_visitor_counts', 'visits_by_day', 
                'poi_cbg', 'visitor_home_cbgs', 'visitor_daytime_cbgs', 'distance_from_home', 'median_dwell',
                'popularity_by_hour', 'popularity_by_day', 'normalized_visits_by_state_scaling'
               ]
safegraph_ma = safegraph_ma[columns_keep]

# Erase the duplicated columns
safegraph_ma = safegraph_ma.loc[:,~safegraph_ma.columns.duplicated()]

# Delete the null rows
safegraph_ma = safegraph_ma [safegraph_ma['location_name'].isnull() == False]

#Erase the rows with no naics_code
safegraph_ma = safegraph_ma[safegraph_ma.naics_code.notnull()].reset_index().drop('index', axis = 1)

#Creating a naics category column (two first digits of naics code)
safegraph_ma['naics_category'] = safegraph_ma['naics_code'].astype(str).str[:2].astype(int)

# Decide which to keep
safegraph_ma = safegraph_ma[['placekey', 'naics_code', 'naics_category', 'raw_visit_counts', 'poi_cbg', 'top_category', 'sub_category', 'location_name']]
safegraph_ma.head(2)

,placekey,naics_code,naics_category,raw_visit_counts,poi_cbg,top_category,sub_category,location_name
0,zzy-222@65y-y8f-kpv,611310.0,61,216.0,2.500390e+11,"Colleges, Universities, and Professional Schools","Colleges, Universities, and Professional Schools",Berkshire Community College
1,zzy-222@62j-ynz-249,442110.0,44,36.0,2.500101e+11,Furniture Stores,Furniture Stores,Mill Stores


In [ ]:
safegraph_ma.to_csv('drive/MyDrive/Amenities/data/safegraph_ma.csv', index = False)